In [1]:
import pandas as pd
import numpy as np

# for visualization
import matplotlib.pyplot as plt
import seaborn as sns

from scipy.sparse import hstack

# to check spelling errors
from textblob import TextBlob

# for text preprocessing
from nltk.tokenize import word_tokenize, sent_tokenize
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords

In [2]:
#! pip install textblob

In [3]:
# to apply bag of words on description and skills column - skills pr bhi i think bow would work better because each row is associated with many skill tags
from sklearn.feature_extraction.text import CountVectorizer


In [4]:
df = pd.read_csv("final_data.csv")
df.head()

,company_name,location,job_title,job_description,rating,employer_estimate,min_salary,max_salary,avg_salary,Size,Founded,Type,Industry,Sector,Revenue,skills,age,seniority,job_simp
0,Procter & Gamble,Mumbai,senior manager – sap security governance risk ...,The Senior Manager’s role will ensure that the...,4.1,0,400000,700000,550000.0,10000+ Employees,1837,Company - Public,Consumer Product Manufacturing,Manufacturing,$10+ billion (USD),"SAP, SOX, Analysis skills, SAP S/4HANA, RMF",187.0,2.0,manager
1,S&P Global,Gurgaon,apprentice – data analyst,We are responsible for assuring that data impa...,4.1,0,500000,700000,600000.0,10000+ Employees,1860,Company - Public,Research and development,Management and consulting,$10+ billion (USD),"Business intelligence, Relational databases, ...",164.0,0.0,analyst
2,Oracuz Infotech Pvt Ltd,Remote,data science intern,You will be assigned with module projects and ...,NaN,1,5000,5000,5000.0,NaN,NaN,NaN,NaN,NaN,NaN,"Web development, Machine learning, Data science",NaN,0.0,data scientist
3,Optum,Bengaluru,manager data scientist,"As an Associate Manager, you will play a criti...",NaN,0,1000000,1000000,1000000.0,NaN,NaN,NaN,NaN,NaN,NaN,"TensorFlow, Big data, SQL, Analysis skills, M...",NaN,2.0,manager data scientist
4,ITI Data,Chennai,aws glue data engineer,Require Financial Services industry experience...,4.2,0,200000,900000,550000.0,201 to 500 Employees,1999,Company - Private,Information Technology Support Services,Information Technology,Unknown / Non-Applicable,"Oracle, XML, Spark, NoSQL, MongoDB",25.0,1.0,data engineer


In [5]:
df.columns

Index(['company_name', 'location', 'job_title', 'job_description', 'rating',
       'employer_estimate', 'min_salary', 'max_salary', 'avg_salary', 'Size',
       'Founded', 'Type', 'Industry', 'Sector', 'Revenue', 'skills', 'age',
       'seniority', 'job_simp'],
      dtype='object')

In [6]:
# dropping columns that are correlated to others or are just representing the same thing as already represented by some other column
newdf = df.drop(columns = ['min_salary', 'max_salary', 'Founded', 'job_title'])

In [7]:
catcols = newdf.select_dtypes("object")

In [8]:
catcols.columns

Index(['company_name', 'location', 'job_description', 'Size', 'Type',
       'Industry', 'Sector', 'Revenue', 'skills', 'job_simp'],
      dtype='object')

In [9]:
numcols = newdf.select_dtypes("number")

In [10]:
numcols.columns

Index(['rating', 'employer_estimate', 'avg_salary', 'age', 'seniority'], dtype='object')

In [11]:
catcols.head()

,company_name,location,job_description,Size,Type,Industry,Sector,Revenue,skills,job_simp
0,Procter & Gamble,Mumbai,The Senior Manager’s role will ensure that the...,10000+ Employees,Company - Public,Consumer Product Manufacturing,Manufacturing,$10+ billion (USD),"SAP, SOX, Analysis skills, SAP S/4HANA, RMF",manager
1,S&P Global,Gurgaon,We are responsible for assuring that data impa...,10000+ Employees,Company - Public,Research and development,Management and consulting,$10+ billion (USD),"Business intelligence, Relational databases, ...",analyst
2,Oracuz Infotech Pvt Ltd,Remote,You will be assigned with module projects and ...,NaN,NaN,NaN,NaN,NaN,"Web development, Machine learning, Data science",data scientist
3,Optum,Bengaluru,"As an Associate Manager, you will play a criti...",NaN,NaN,NaN,NaN,NaN,"TensorFlow, Big data, SQL, Analysis skills, M...",manager data scientist
4,ITI Data,Chennai,Require Financial Services industry experience...,201 to 500 Employees,Company - Private,Information Technology Support Services,Information Technology,Unknown / Non-Applicable,"Oracle, XML, Spark, NoSQL, MongoDB",data engineer


In [12]:
catcols.isnull().sum()

company_name        0
location            0
job_description     0
Size               51
Type               51
Industry           51
Sector             51
Revenue            51
skills              1
job_simp            0
dtype: int64

### Lets try bringing a bit bias to the system - filling null values in each column with random values from the respective column

In [14]:
catcols['Size'].dropna().sample(1, random_state = 12).iloc[0]

'10000+ Employees'

In [15]:
catcols['Size'].fillna(catcols['Size'].dropna().sample(1).values[0])

0            10000+ Employees
1            10000+ Employees
2            10000+ Employees
3            10000+ Employees
4        201 to 500 Employees
                ...          
369    1001 to 5000 Employees
370         1 to 50 Employees
371          10000+ Employees
372         1 to 50 Employees
373          10000+ Employees
Name: Size, Length: 374, dtype: object

In [16]:
for col in catcols.columns:
    catcols[col] = catcols[col].fillna(catcols[col].dropna().sample(1, random_state = 12).iloc[0]) # .iloc[0] or .values[0] just extracts the required part from the series got

In [17]:
catcols

,company_name,location,job_description,Size,Type,Industry,Sector,Revenue,skills,job_simp
0,Procter & Gamble,Mumbai,The Senior Manager’s role will ensure that the...,10000+ Employees,Company - Public,Consumer Product Manufacturing,Manufacturing,$10+ billion (USD),"SAP, SOX, Analysis skills, SAP S/4HANA, RMF",manager
1,S&P Global,Gurgaon,We are responsible for assuring that data impa...,10000+ Employees,Company - Public,Research and development,Management and consulting,$10+ billion (USD),"Business intelligence, Relational databases, ...",analyst
2,Oracuz Infotech Pvt Ltd,Remote,You will be assigned with module projects and ...,10000+ Employees,Company - Public,Research and development,Management and consulting,$10+ billion (USD),"Web development, Machine learning, Data science",data scientist
3,Optum,Bengaluru,"As an Associate Manager, you will play a criti...",10000+ Employees,Company - Public,Research and development,Management and consulting,$10+ billion (USD),"TensorFlow, Big data, SQL, Analysis skills, M...",manager data scientist
4,ITI Data,Chennai,Require Financial Services industry experience...,201 to 500 Employees,Company - Private,Information Technology Support Services,Information Technology,Unknown / Non-Applicable,"Oracle, XML, Spark, NoSQL, MongoDB",data engineer
...,...,...,...,...,...,...,...,...,...,...
369,The Trade Desk,Bengaluru,Have a firm grasp on basic data structures and...,1001 to 5000 Employees,Company - Public,Internet & Web Services,Information Technology,$2 to $5 billion (USD),"Machine learning, Distributed systems",intern
370,SmartHelio,Delhi,Join our team as an experienced data scientist...,1 to 50 Employees,Unknown,--,--,Unknown / Non-Applicable,"Computer science, Alteryx, English, Tableau, SQL",data scientist
371,RP2,India,Experiences you gain in this internship will h...,10000+ Employees,Company - Public,Research and development,Management and consulting,$10+ billion (USD),"Image processing, Machine learning, Natural l...",data scientist
372,Recruitment Smart,Ahmedabad,Job Title : Data Science Intern Created on: 24...,1 to 50 Employees,Company - Private,Enterprise Software & Network Solutions,Information Technology,Unknown / Non-Applicable,Data science,data scientist


##### if we can apply countvectoirizer directly on whole df with multiple cat cols or cat plus num cols since will it create vocabulary picking unique words from all the columns in the df or will correctly do col by col each ?

- CountVectorizer is typically applied to a single column at a time because it creates a vocabulary based on unique words found in the entire input. If you apply it directly to the entire DataFrame, it will treat each cell across all columns as a single string, creating a vocabulary from all unique words in all cells combined. This may lead to mixing terms from different columns, which isn't ideal for distinct categorical columns.

- To handle each categorical column separately:

- Apply CountVectorizer to each column individually, creating a separate sparse matrix for each one.
Concatenate these sparse matrices horizontally if you want to combine them into a single feature matrix.

#### How features are created if we apply encoding directly without preprocessing

We see that the data needs preprocessing to deal with problems like :
- 'analysis', 'analytics' - counted as different words in the vocab
- strange words like '4hana', 'arcgis', 'drupal'.. etc
- 

In [20]:
vector = CountVectorizer()
vector.fit_transform(catcols.skills).toarray()
print(len(vector.get_feature_names_out()))
vector.get_feature_names_out()

206


array(['4hana', 'access', 'administration', 'administrative', 'adobe',
       'adwords', 'agile', 'ai', 'ajax', 'alteryx', 'analysis',
       'analytics', 'android', 'angular', 'apache', 'apis', 'application',
       'applications', 'arcgis', 'architecture', 'asp', 'asset',
       'assurance', 'authentication', 'automation', 'aws', 'azure',
       'basic', 'bi', 'big', 'bloomberg', 'bootstrap', 'business',
       'cassandra', 'cd', 'chain', 'ci', 'clerical', 'clinical', 'cloud',
       'communication', 'computer', 'computing', 'content', 'creative',
       'css', 'cucumber', 'customer', 'cybersecurity', 'data', 'database',
       'databases', 'deep', 'deployment', 'design', 'development',
       'devops', 'distributed', 'django', 'docker', 'driven', 'drupal',
       'elasticsearch', 'electrical', 'encryption', 'engineering',
       'engines', 'english', 'enterprise', 'entry', 'erp', 'excel',
       'experience', 'financial', 'ftp', 'git', 'go', 'google', 'gradle',
       'hadoop', 'han

In [21]:
# here we note that our description also has skills attached to it ... we do not want that since its already separated to a new column
# also note that we need to remove the truncating 3 dots after every description 
catcols.loc[1,'job_description']

'We are responsible for assuring that data impacts are evaluate, understood, and communicated across all teams so that business continuity is maintained.…\r\nSkills: Business intelligence, Relational databases, Tableau, Databases, SQL'

In [22]:
catcols.loc[1,'job_description'].split("\n")[0]

'We are responsible for assuring that data impacts are evaluate, understood, and communicated across all teams so that business continuity is maintained.…\r'

In [23]:
for i in range(len(catcols.job_description)):
    catcols.loc[i,'job_description'] = catcols.job_description[i].split("\n")[0]

In [24]:
import re
def remove_trailing_dots(text):
    return re.sub(r'\s*\.*\s*$|[.…]+\s*$', '', text)

catcols['job_description'] = catcols.job_description.apply(remove_trailing_dots)

### Text Preprocessing

In [26]:
# Lowercasing
for col in catcols.columns:
    catcols[col] = catcols[col].apply(str.lower)

In [27]:
def remove_punctuation(text):
    import string
    withoutpunc = [x for x in text if x not in string.punctuation]
    return ''.join(withoutpunc)

In [28]:
remove_punctuation(catcols.skills[0])

' sap sox analysis skills sap s4hana rmf'

In [29]:
catcols.skills[0]

' sap, sox, analysis skills, sap s/4hana, rmf'

In [30]:
# removing punctuations
for col in catcols.columns:
    catcols[col] = catcols[col].apply(remove_punctuation)

In [31]:
txblob = TextBlob('incorr ttexs').correct().string

In [32]:
txblob

'income steps'

In [33]:
# # correct spelling correction
# for col in catcols.columns:
#     catcols[col] = catcols[col].apply(lambda x : TextBlob(str(x)).correct().string)

##### few words we know like machine learning, deep learning data science need to be counted together 
we can think of doing something for this later, before applying count vectorizer

##### Removing stop words
We'll handle stop words inside countvectorizer itself

In [36]:
# import nltk
# nltk.download('stopwords')

In [37]:
# stopwords.words('english')

In [38]:
# from nltk.corpus import stopwords
# stopwords.words('english')

# custom function to remove stopwords (later we'll see that we can even use count vectorizer to remove stopwords
def rem_stopwords(text):
    filteredwords = [word for word in text.split() if word not in stopwords.words('english')]
    return " ".join(filteredwords)

In [39]:
catcols['job_description'][0]

'the senior manager’s role will ensure that the sap security strategy and risk and compliance management frameworks are always uptodate new threats are'

In [40]:
rem_stopwords(catcols['job_description'][0])

'senior manager’s role ensure sap security strategy risk compliance management frameworks always uptodate new threats'

##### we can remove this type of words are…… from the text from all columns using regex

In [42]:
# import re

# # Define a function to remove words associated with repeated dots (or similar patterns)
# def remove_words_with_dots(text):
#     # Replace any word followed by three or more dots with an empty string
#     return re.sub(r'\b\w+\.\.\.\.+', '', text)

# # Apply the function to all columns in the DataFrame
# for col in catcols.columns:
#     print( catcols['job_description'].astype(str).apply(remove_words_with_dots))
#     print(remove_words_with_dots(catcols['job_description'][0]))
#     break


##### Applying lemmatization

- Stemming: May result in non-valid words, as it focuses on removing prefixes and suffixes to obtain a common root. 
- Lemmatization: Ensures the output is a valid word, considering the context and part of speech.
- WordNet - A lexical database for English used for lemmatization and synonym detection.

In [45]:
# import nltk
# nltk.download('wordnet')

In [46]:
example = "the senior manager’s role will ensure"
" ".join([WordNetLemmatizer().lemmatize(word) for word in example.split()])

'the senior manager’s role will ensure'

In [47]:
lemmatizer = WordNetLemmatizer()
def lemmatize(text):
    return " ".join([lemmatizer.lemmatize(word) for word in text.split()])


In [48]:
for col in catcols.columns:
    catcols[col] = catcols[col].apply(lemmatize)

### Text Encoding

In [104]:
# Bag of words
vector = CountVectorizer()
arr = vector.fit_transform(catcols.skills).toarray()
print(arr)
print(len(vector.get_feature_names_out()))
print(arr.shape)
vector.get_feature_names_out()

[[0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 1 0]
 ...
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]]
200
(374, 200)


array(['access', 'administration', 'administrative', 'adobe', 'adwords',
       'agile', 'ai', 'ajax', 'alteryx', 'analysis', 'analytics',
       'android', 'angular', 'apache', 'apis', 'application', 'arcgis',
       'architecture', 'aspnet', 'asset', 'assurance', 'authentication',
       'automation', 'aws', 'azure', 'basic', 'bi', 'big', 'bloomberg',
       'bootstrap', 'business', 'cassandra', 'chain', 'cicd', 'clerical',
       'clinical', 'cloud', 'communication', 'computer', 'computing',
       'content', 'creative', 'cs', 'cucumber', 'customer',
       'cybersecurity', 'data', 'database', 'deep', 'deployment',
       'design', 'development', 'devops', 'distributed', 'django',
       'docker', 'drupal', 'elasticsearch', 'electrical', 'encryption',
       'engine', 'engineering', 'english', 'enterprise', 'entry', 'erp',
       'excel', 'experience', 'financial', 'ftp', 'git', 'go', 'google',
       'gradle', 'hadoop', 'hana', 'hive', 'image', 'incident',
       'information', 'in

In [106]:
pd.DataFrame(arr, columns = vector.get_feature_names_out())

,access,administration,administrative,adobe,adwords,agile,ai,ajax,alteryx,analysis,...,training,troubleshooting,tuning,uml,vendor,vision,visual,visualization,web,xml
0,0,0,0,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
3,0,0,0,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
369,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
370,0,0,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
371,0,0,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
372,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [108]:
encoded_dfs = []  # List to store encoded DataFrames

for col in catcols.columns:
    vector = CountVectorizer()
    arr = vector.fit_transform(catcols[col]).toarray()  # Use each column dynamically
    # Create a DataFrame with feature names as column names
    encoded_df = pd.DataFrame(arr, columns=[f"{col}_{feat}" for feat in vector.get_feature_names_out()])
    encoded_dfs.append(encoded_df)  # Append the DataFrame to the list

In [112]:
combined_df = pd.concat(encoded_dfs, axis = 1)

In [114]:
combined_df

,company_name_a66,company_name_adci,company_name_adobe,company_name_airtel,company_name_alignerr,company_name_altair,company_name_american,company_name_amex,company_name_analytics,company_name_ananta,...,job_simp_intern,job_simp_learning,job_simp_machine,job_simp_manager,job_simp_nlp,job_simp_other,job_simp_python,job_simp_related,job_simp_scientist,job_simp_software
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,1,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
369,0,0,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
370,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
371,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
372,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0


#### Removing null values from numeric columns as well

In [120]:
numcols.isnull().sum()

rating                53
employer_estimate      0
avg_salary             0
age                  106
seniority              0
dtype: int64

In [134]:
numcols['rating'].dropna().sample().values[0]

4.2

In [140]:
numcols['rating'].fillna(numcols['rating'].dropna().sample(random_state = 14).values[0])

0      4.1
1      4.1
2      4.2
3      4.2
4      4.2
      ... 
369    3.9
370    4.7
371    4.2
372    4.3
373    4.2
Name: rating, Length: 374, dtype: float64

In [144]:
for col in numcols.columns:
    numcols[col] = numcols[col].fillna(numcols[col].dropna().sample(1, random_state = 12).values[0])

In [146]:
numcols

,rating,employer_estimate,avg_salary,age,seniority
0,4.1,0,550000.0,187.0,2.0
1,4.1,0,600000.0,164.0,0.0
2,4.1,1,5000.0,61.0,0.0
3,4.1,0,1000000.0,61.0,2.0
4,4.2,0,550000.0,25.0,1.0
...,...,...,...,...,...
369,3.9,0,-1.0,15.0,0.0
370,4.7,0,-1.0,61.0,1.0
371,4.1,0,-1.0,61.0,0.0
372,4.3,0,-1.0,9.0,0.0


####  Merging catcols and numcols together 

In [153]:
final_df = pd.concat([combined_df, numcols], axis = 1)

In [164]:
final_df

,company_name_a66,company_name_adci,company_name_adobe,company_name_airtel,company_name_alignerr,company_name_altair,company_name_american,company_name_amex,company_name_analytics,company_name_ananta,...,job_simp_other,job_simp_python,job_simp_related,job_simp_scientist,job_simp_software,rating,employer_estimate,avg_salary,age,seniority
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,4.1,0,550000.0,187.0,2.0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,4.1,0,600000.0,164.0,0.0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,1,0,4.1,1,5000.0,61.0,0.0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,1,0,4.1,0,1000000.0,61.0,2.0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,4.2,0,550000.0,25.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
369,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,3.9,0,-1.0,15.0,0.0
370,0,0,0,0,0,0,0,0,0,0,...,0,0,0,1,0,4.7,0,-1.0,61.0,1.0
371,0,0,0,0,0,0,0,0,0,0,...,0,0,0,1,0,4.1,0,-1.0,61.0,0.0
372,0,0,0,0,0,0,0,0,0,0,...,0,0,0,1,0,4.3,0,-1.0,9.0,0.0


##### it has a very high dimensionality

In [160]:
final_df.columns 

Index(['company_name_a66', 'company_name_adci', 'company_name_adobe',
       'company_name_airtel', 'company_name_alignerr', 'company_name_altair',
       'company_name_american', 'company_name_amex', 'company_name_analytics',
       'company_name_ananta',
       ...
       'job_simp_other', 'job_simp_python', 'job_simp_related',
       'job_simp_scientist', 'job_simp_software', 'rating',
       'employer_estimate', 'avg_salary', 'age', 'seniority'],
      dtype='object', length=1674)

#### Exporting the preprocessed df 

In [174]:
final_df.to_csv("Bag_of_words_data.csv", index = False)

### Note that in this sheet we have used Count_vectorizer i.e. BOW on all of the categorical columns separately .